In [1]:
from QuantumCircuit import QuantumCircuit as nqc
import pickle
import json
import numpy as np

In [2]:
noise = pickle.load(open("noise_data_2025_04_23.pkl", "rb"))
noise = json.JSONDecoder().decode(json.dumps(noise))

In [3]:
circ = nqc(num_qubits=3, noise_model=noise, num_trajectories=100, num_cores=5, threshold=1e-4)

Completed Extraction of ECR Errors.
Starting post-processing on Single Qubit Errors.
Completed post-processing on Single Qubit Errors.
Processing ECR errors.
  Qubit pair (0, 1): 20/144 errors above threshold (124 filtered out)
  Qubit pair (1, 2): 20/144 errors above threshold (124 filtered out)
ECR errors processed.
Building Noise Operators for ECR Errors.
Completed building Noise Operators for ECR Errors.
Extracting Measurement Errors.
Available qubits in roerror_map: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126

In [4]:
gate_set_single = {"x": circ.X, "y":circ.Y, "z":circ.Z, "h":circ.H, "sx":circ.SX, "rx":circ.RX, "ry":circ.RY, "rz":circ.RZ}
gate_set_double = [circ.CX, circ.CY, circ.CZ]

def build_random_circuit(depth):
    for _ in range(depth):
        for i in range(3):
            choice = np.random.choice(list(gate_set_single.keys()))
        if choice in ["rx", "ry", "rz"]:
            angle = np.random.uniform(0, 2*np.pi)
            gate_set_single[choice](angle, i)
        else:
            gate_set_single[choice](i)
        for i in range(2):
            np.random.choice(gate_set_double)(i, i+1)
        np.random.choice(gate_set_double)(2, 0)

In [5]:
depth = 20
build_random_circuit(depth)

In [ ]:
p_mcwf = circ.execute(qubits=[0,1,2], num_trajectories=100)

In [ ]:
p_pure = circ.run_pure_state([0,1,2])

In [ ]:
p_ref = circ.run_with_density_matrix([0,1,2])

In [ ]:
p_mcwf1k = circ.execute(qubits=[0,1,2], num_trajectories=1000)

In [ ]:
def metrics(p1,p2):
    coeff = np.sum(np.sqrt(p1 * p2))
    dist = -np.log(coeff)
    hellinger = np.sqrt(1 - coeff)
    m = 0.5 * (p1 + p2)
    js = 0.5 * np.sum(p1 * np.log(1e-10 + p1 / m) + p2 * np.log(1e-10 + p2 / m))
    print(f"Coefficient: {coeff}, Distance: {dist}, Hellinger: {hellinger}, Jensen-Shannon: {js}")

In [ ]:
print("Metrics for MCWF (100 Trajectories):")
metrics(p_mcwf, p_ref)
print("====================================================")
print("Metrics for MCWF (1000 Trajectories):")
metrics(p_mcwf1k, p_ref)
print("====================================================")
print("Metrics for Pure State:")
metrics(p_pure, p_ref)


Metrics for MCWF:
Coefficient: 0.9838768906516524, Distance: 0.01625450088330295, Hellinger: 0.12697680634016434, Jensen-Shannon: 0.015786383647639318
Metrics for Pure State:
Coefficient: 0.9110122807171013, Distance: 0.09319890133323577, Hellinger: 0.2983080945648286, Jensen-Shannon: 0.062924397465392


In [12]:
len(circ.instruction_list)

655